<a href="https://colab.research.google.com/github/jpellicott/ner_and_clustering/blob/main/fine_tune_bert_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install NERDA


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForTokenClassification
from tqdm.auto import tqdm
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_data_path = "/content/drive/MyDrive/glg_data/ner.csv"

In [ ]:
data = pd.read_csv(
    training_data_path, encoding = "ISO-8859-1", error_bad_lines=False, usecols=['sentence_idx', 'word', 'tag']
)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data['tag'].value_counts()

O                  889973
B-geo               37525
B-tim               20193
B-org               20184
I-per               17382
B-per               17011
I-org               16537
B-gpe               16392
I-geo                7409
I-tim                6298
B-art                 434
B-eve                 348
I-eve                 297
I-art                 280
I-gpe                 229
B-nat                 226
I-nat                  76
prev-prev-lemma         1
Name: tag, dtype: int64

In [ ]:
data

,sentence_idx,word,tag
0,1,Thousands,O
1,1,of,O
2,1,demonstrators,O
3,1,have,O
4,1,marched,O
...,...,...,...
1050791,47959,they,O
1050792,47959,responded,O
1050793,47959,to,O
1050794,47959,the,O


In [ ]:
data  = data[data['sentence_idx'] != 'prev-lemma'].dropna(subset=['sentence_idx']).reset_index(drop=True)
data['sentence_idx'] = data['sentence_idx'].astype(int)
split_thresh = data['sentence_idx'].max() * 0.9
df_train, df_valid = data[data['sentence_idx'] < split_thresh], data[data['sentence_idx'] >= split_thresh]
len(df_train), len(df_valid)

(946852, 103942)

In [ ]:
agg_func = lambda s: [ [w,t] for w,t in zip(s["word"].values.tolist(),s["tag"].values.tolist())]

x_train_grouped = df_train.groupby("sentence_idx").apply(agg_func)
x_valid_grouped = df_valid.groupby("sentence_idx").apply(agg_func)

x_train_sentences = [[s[0] for s in sent] for sent in x_train_grouped.values]
x_valid_sentences = [[s[0] for s in sent] for sent in x_valid_grouped.values]

x_train_tags = [[t[1] for t in tag] for tag in x_train_grouped.values]
x_valid_tags = [[t[1] for t in tag] for tag in x_valid_grouped.values]

In [ ]:
training_data = {}
validation_data = {}

In [ ]:
training_data["sentences"] = (x_train_sentences)
training_data["tags"] = (x_train_tags)

validation_data["sentences"] = (x_valid_sentences)
validation_data["tags"] = (x_valid_tags)

In [ ]:
tag_scheme = [
'B-geo',
'B-tim',
'B-org',
'I-per',
'B-per',
'I-org',
'B-gpe',
'I-geo',
'I-tim',
'B-art',
'B-eve',
'I-eve',
'I-art',
'I-gpe',
'B-nat',
'I-nat'

]

In [ ]:
transformer = 'dslim/bert-base-NER'


In [ ]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
'epochs' : 4,
'warmup_steps' : 500,
'train_batch_size': 12,
'learning_rate': 0.0001
}

In [ ]:
from NERDA.models import NERDA
model = NERDA(
dataset_training = training_data,
dataset_validation = validation_data,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = transformer,
dropout = dropout,
hyperparameters = training_hyperparameters,
max_len = 256
)

Device automatically set to: cuda


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,



 Epoch 1 / 4


100%|██████████| 600/600 [00:40<00:00, 14.81it/s]


Train Loss = 0.25770874061484894 Valid Loss = 0.17790488844368763

 Epoch 2 / 4


100%|██████████| 600/600 [00:40<00:00, 14.82it/s]


...

100%|██████████| 600/600 [00:40<00:00, 14.83it/s]


Train Loss = 0.0936225492502886 Valid Loss = 0.17741833143115704

 Epoch 4 / 4


100%|██████████| 600/600 [00:40<00:00, 14.84it/s]

Train Loss = 0.055826972426267905 Valid Loss = 0.1997692537784193


'Model trained successfully'

In [ ]:
model.evaluate_performance(validation_data)

/usr/local/lib/python3.7/dist-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 2 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 2 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded le

,Level,F1-Score,Precision,Recall
0,B-geo,0.868048,0.847592,0.889516
1,B-tim,0.903258,0.913174,0.893555
2,B-org,0.723456,0.753815,0.695448
3,I-per,0.884504,0.861504,0.908766
4,B-per,0.838112,0.826753,0.849788
5,I-org,0.734023,0.784645,0.689537
6,B-gpe,0.953299,0.953586,0.953012
7,I-geo,0.781525,0.779240,0.783824
8,I-tim,0.820639,0.872822,0.774343
9,B-art,0.268657,0.375000,0.209302


In [32]:
torch.save(model, "/content/drive/MyDrive/glg/ner_model_nerda.pth")


In [33]:
model = torch.load('/content/drive/MyDrive/glg/ner_model_nerda.pth')

In [39]:
model.predict_text("""Nebraska Sandhills wagyu beef ranch and processing plant bought by private equity firm. The firm plans on adding eco tourism to the ranch's operations""")

([['Nebraska',
   'Sandhills',
   'wagyu',
   'beef',
   'ranch',
   'and',
   'processing',
   'plant',
   'bought',
   'by',
   'private',
   'equity',
   'firm',
   '.'],
  ['The',
   'firm',
   'plans',
   'on',
   'adding',
   'eco',
   'tourism',
   'to',
   'the',
   'ranch',
   "'s",
   'operations']],
 [['B-org',
   'I-org',
   'I-org',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'],
  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']])